In [1]:
from __future__ import print_function

import tensorflow as tf
import pandas as pd

from tensorflow.contrib.learn import learn_runner

In [17]:
TRAIN_FILE = 'D:\Study\Ostfold\MachineLearning\git\data\ohenc_data_colNames_tensorflow.train';
TEST_FILE = 'D:\Study\Ostfold\MachineLearning\git\data\ohenc_data_colNames_tensorflow.test';

redundant_label = 'outcome<50K'
label_name = 'outcome>50K'

train = pd.read_table(TRAIN_FILE, sep=' ')
train.pop(redundant_label)

train_x, train_y = train, train.pop(label_name)
display(train_x.head(5))
display(train_y.head(5))

# small sample data, training model should overfit this dataset
sample_train_x = train_x.loc[[0,50],:]
sample_train_y = train_y.loc[[0,50]]


test = pd.read_table(TEST_FILE, sep=' ')
test.pop(redundant_label)

test_x, test_y = test, test.pop(label_name)

,age,workclassMissing,workclassFederal-gov,workclassLocal-gov,workclassNever-worked,workclassPrivate,workclassSelf-emp-inc,workclassSelf-emp-not-inc,workclassState-gov,workclassWithout-pay,...,native-countryPortugal,native-countryPuerto-Rico,native-countryScotland,native-countrySouth,native-countryTaiwan,native-countryThailand,native-countryTrinadad-Tobago,native-countryUnited-States,native-countryVietnam,native-countryYugoslavia
0,39,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,50,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,38,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,53,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,28,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


0    0
1    0
2    0
3    0
4    0
Name: outcome>50K, dtype: int64

In [18]:
# Parameters
learning_rate = 0.0001
num_steps = 10000
batch_size = 256
display_step = 100

# Network Parameters
n_hidden_1 = 1024 # 1st layer number of neurons
n_hidden_2 = 1024 # 2nd layer number of neurons
n_hidden_3 = 512 # 1st layer number of neurons
n_hidden_4 = 256 # 2nd layer number of neurons
num_input = 108 # data input 
num_classes = 2 #  total classes

In [19]:
# Define the input function for training
def train_input_fn(features, labels, batch_size):
    """An input function for training"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle, repeat, and batch the examples.
    # TODO check values of following parameters
    dataset = dataset.shuffle(100000).repeat().batch(batch_size)

    # Return the dataset.
    return dataset.make_one_shot_iterator().get_next()

In [20]:
def eval_input_fn(features, labels, batch_size):
    """An input function for evaluation or prediction"""
    features=dict(features)
    if labels is None:
        # No labels, use only features.
        inputs = features
    else:
        inputs = (features, labels)

    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices(inputs)

    # Batch the examples
    assert batch_size is not None, "batch_size must not be None"
    dataset = dataset.batch(batch_size)

    # Return the dataset.
    return dataset.make_one_shot_iterator().get_next()

In [21]:
def custom_model2_fn(features, labels, mode, params):
    """DNN with three hidden layers, and dropout of 0.1 probability."""
    # Create three fully connected layers each layer having a dropout
    # probability of 0.1.
    net = tf.feature_column.input_layer(features, params['feature_columns'])
    
    net = tf.layers.batch_normalization(net, training = (mode == tf.estimator.ModeKeys.TRAIN))
    for units in params['hidden_units']:
        net = tf.layers.dense(net, units=units)
        net = tf.layers.batch_normalization(net, training = (mode == tf.estimator.ModeKeys.TRAIN))
        net = tf.layers.dropout(net, training = (mode == tf.estimator.ModeKeys.TRAIN), rate=0.5)
        net = tf.nn.relu(net)

    # Compute logits (1 per class).
    logits = tf.layers.dense(net, params['n_classes'], activation=None)

    # Compute predictions.
    predicted_classes = tf.argmax(logits, 1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_ids': predicted_classes[:, tf.newaxis],
            'probabilities': tf.nn.softmax(logits),
            'logits': logits,
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # Compute loss.
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    
#     onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=2)
#     loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)

    # Compute evaluation metrics.
    accuracy = tf.metrics.accuracy(labels=labels,
                                   predictions=predicted_classes,
                                   name='acc_op')
    metrics = {'accuracy': accuracy}
    tf.summary.scalar('accuracy', accuracy[1])

    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode, loss=loss, eval_metric_ops=metrics)

    # Create training op.
    assert mode == tf.estimator.ModeKeys.TRAIN

    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    # what is global_step
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
#     logging_hook = tf.train.LoggingTensorHook({"loss" : loss, "accuracy" : accuracy}, every_n_iter=100)
    
    
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

In [27]:
# Feature columns describe how to use the input.
my_feature_columns = []
for key in train_x.keys():
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))
    
# Build the Estimator
# model = tf.estimator.Estimator(
#     custom_model1_fn,
#     params={'feature_columns': my_feature_columns,})

# Build 2 hidden layer DNN with 10, 10 units respectively.
custom_classifier = tf.estimator.Estimator(
    model_fn=custom_model2_fn,
    params={
        'feature_columns': my_feature_columns,
        # Two hidden layers of 10 nodes each.
        #'hidden_units': [n_hidden_1, n_hidden_2, n_hidden_3, n_hidden_4],
        'hidden_units': [1024, 256],
        # The model must choose between 3 classes.
        'n_classes': 2,
    })


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_is_chief': True, '_model_dir': 'C:\\Users\\ADMIN\\AppData\\Local\\Temp\\tmpvhp7v49j', '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_tf_random_seed': None, '_service': None, '_master': '', '_num_ps_replicas': 0, '_log_step_count_steps': 100, '_num_worker_replicas': 1, '_session_config': None, '_task_type': 'worker', '_keep_checkpoint_max': 5, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001FE8F407978>, '_task_id': 0}


In [23]:
# Train the small sample model with overfitting expectation
# tf.logging.set_verbosity(tf.logging.INFO)
# custom_classifier.train(lambda:train_input_fn(sample_train_x, sample_train_y, batch_size), steps=num_steps)
# eval_result = custom_classifier.evaluate(
#     input_fn=lambda:eval_input_fn(sample_train_x, sample_train_y, batch_size))
# print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

In [28]:
# Train the Model
# for i in range(100):
custom_classifier.train(lambda:train_input_fn(train_x, train_y, batch_size), steps=1000)
# Evaluate the model.
eval_result = custom_classifier.evaluate(
    input_fn=lambda:eval_input_fn(test_x, test_y, batch_size))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\ADMIN\AppData\Local\Temp\tmpvhp7v49j\model.ckpt.
INFO:tensorflow:step = 1, loss = 0.8592147
INFO:tensorflow:global_step/sec: 13.9795
INFO:tensorflow:step = 101, loss = 0.4469509 (7.157 sec)
INFO:tensorflow:global_step/sec: 11.6347
INFO:tensorflow:step = 201, loss = 0.4339753 (8.597 sec)
INFO:tensorflow:global_step/sec: 11.7585
INFO:tensorflow:step = 301, loss = 0.46708825 (8.505 sec)
INFO:tensorflow:global_step/sec: 9.11904
INFO:tensorflow:step = 401, loss = 0.51096946 (10.967 sec)
INFO:tensorflow:global_step/sec: 15.2777
INFO:tensorflow:step = 501, loss = 0.4066142 (6.544 sec)
INFO:tensorflow:global_step/sec: 11.3141
INFO:tensorflow:step = 601, loss = 0.4211011 (8.840 sec)
INFO:tensorflow:global_step/sec: 10.9926
INFO:tensorflow:step = 701, loss = 0.31240708 (9.097 sec)
INFO:tensorflow:global_step/sec: 10.9206
INFO:tensorflow:step = 801, loss = 0.4007742 (9.158 sec)
INFO:tensorflow:globa

In [25]:
# Evaluate the model.
# Build 2 hidden layer DNN with 10, 10 units respectively.
custom_classifier1 = tf.estimator.Estimator(
    model_fn=custom_model2_fn,
    params={
        'feature_columns': my_feature_columns,
        # Two hidden layers of 10 nodes each.
        #'hidden_units': [n_hidden_1, n_hidden_2, n_hidden_3, n_hidden_4],
        'hidden_units': [256, 256],
        # The model must choose between 3 classes.
        'n_classes': 2,
    })
custom_classifier1.train(lambda:train_input_fn(train_x, train_y, batch_size), steps=1000)
# Evaluate the model.
eval_result = custom_classifier1.evaluate(
    input_fn=lambda:eval_input_fn(test_x, test_y, batch_size))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_is_chief': True, '_model_dir': 'C:\\Users\\ADMIN\\AppData\\Local\\Temp\\tmp9jamuh98', '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_tf_random_seed': None, '_service': None, '_master': '', '_num_ps_replicas': 0, '_log_step_count_steps': 100, '_num_worker_replicas': 1, '_session_config': None, '_task_type': 'worker', '_keep_checkpoint_max': 5, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001FE80254D68>, '_task_id': 0}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\ADMIN\AppData\Local\Temp\tmp9jamuh98\model.ckpt.
INFO:tensorflow:step = 1, loss = 1.0559819
INFO:tensorflow:global_step/sec: 20.7408
INFO:tensorflow:step = 101, loss = 0.5211351 (4.837 sec)
INFO:tensorflow:global_step/sec: 12.8332
INFO:tensorflow:step = 201, loss = 0.46208388 (7.802

In [26]:
# Build 2 hidden layer DNN with 10, 10 units respectively.
custom_classifier2 = tf.estimator.Estimator(
    model_fn=custom_model2_fn,
    params={
        'feature_columns': my_feature_columns,
        # Two hidden layers of 10 nodes each.
        #'hidden_units': [n_hidden_1, n_hidden_2, n_hidden_3, n_hidden_4],
        'hidden_units': [1024, 1024],
        # The model must choose between 3 classes.
        'n_classes': 2,
    })
custom_classifier2.train(lambda:train_input_fn(train_x, train_y, batch_size), steps=1000)
# Evaluate the model.
eval_result = custom_classifier2.evaluate(
    input_fn=lambda:eval_input_fn(test_x, test_y, batch_size))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_is_chief': True, '_model_dir': 'C:\\Users\\ADMIN\\AppData\\Local\\Temp\\tmptxeju068', '_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_tf_random_seed': None, '_service': None, '_master': '', '_num_ps_replicas': 0, '_log_step_count_steps': 100, '_num_worker_replicas': 1, '_session_config': None, '_task_type': 'worker', '_keep_checkpoint_max': 5, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001FE81174080>, '_task_id': 0}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into C:\Users\ADMIN\AppData\Local\Temp\tmptxeju068\model.ckpt.
INFO:tensorflow:step = 1, loss = 0.7588783
INFO:tensorflow:global_step/sec: 7.69815
INFO:tensorflow:step = 101, loss = 0.4418798 (12.990 sec)
INFO:tensorflow:global_step/sec: 7.72497
INFO:tensorflow:step = 201, loss = 0.44012108 (12.9